# Bedrock Mistral
reference: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-mistral.html

In [30]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

  Using cached boto3-1.34.84-py3-none-any.whl.metadata (6.6 kB)
  Using cached awscli-1.32.84-py3-none-any.whl.metadata (11 kB)
  Using cached botocore-1.34.84-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl.metadata (14 kB)
  Using cached rsa-4.7.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached pyasn1-0.6.0-py2.py3-none-any.whl.metadata (8.3 kB)
Using cached boto3-1.34.84-py3-none-any.whl (139 kB)
Using cached awscli-1.32.84-py3-none-any.whl (4.4 MB)
Using cached botoco

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aws-langchain 0.0.1 requires langchain==0.0.137, but you have langchain 0.0.309 which is incompatible.
streamlit 1.25.0 requires pillow<10,>=7.1.0, but you have pillow 10.0.0 which is incompatible.
wasabi 1.1.2 requires colorama>=0.4.6; sys_platform == "win32" and python_version >= "3.7", but you have colorama 0.4.4 which is incompatible.


This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install [LangChain](https://github.com/hwchase17/langchain):

In [31]:
%pip install --quiet langchain==0.0.309

Note: you may need to restart the kernel to use updated packages.


In [32]:
%pip install termcolor --quiet

Note: you may need to restart the kernel to use updated packages.


---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).

#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.

#### Use a different AWS Region

If you're running this notebook from your own computer or a SageMaker notebook in a different AWS Region from where Bedrock is set up, you can un-comment the `os.environ['AWS_DEFAULT_REGION']` line below and specify the region to use.

#### Use a specific profile

In case you're running this notebook from your own computer where you have setup the AWS CLI with multiple profiles, and the profile which has access to Bedrock is not the default one, you can un-comment the `os.environ['AWS_PROFILE']` line below and specify the profile to use.

#### Use a different role

In case you or your company has setup a specific, separate [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) to access Bedrock, you can specify it by un-commenting the `os.environ['BEDROCK_ASSUME_ROLE']` line below. Ensure that your current user or role have permissions to [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) such role.

#### A note about `langchain`

The Bedrock classes provided by `langchain` create a Bedrock boto3 client by default. To customize your Bedrock configuration, we recommend to explicitly create the Bedrock client using the method below, and pass it to the [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) class instantiation method using `client=bedrock_runtime`

In [33]:
import boto3
import json 
from termcolor import colored  

bedrock = boto3.client(service_name="bedrock", region_name="us-west-2")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use 

In [34]:
response = bedrock.list_foundation_models()
# Filter models with modelArn containing "mistral"
filtered_models = [model for model in response['modelSummaries'] if 'mistral' in model['modelArn'] ]

# Print the filtered models
for model in filtered_models:
    print("Model Name:", model['modelName'])
    print("Model ARN:", model['modelArn'])
    print("Provider Name:", model['providerName'])
    print()




Model Name: Mistral 7B Instruct
Model ARN: arn:aws:bedrock:us-west-2::foundation-model/mistral.mistral-7b-instruct-v0:2
Provider Name: Mistral AI

Model Name: Mixtral 8x7B Instruct
Model ARN: arn:aws:bedrock:us-west-2::foundation-model/mistral.mixtral-8x7b-instruct-v0:1
Provider Name: Mistral AI

Model Name: Mistral Large
Model ARN: arn:aws:bedrock:us-west-2::foundation-model/mistral.mistral-large-2402-v1:0
Provider Name: Mistral AI



---

## Common inference parameter definitions

### Randomness and Diversity

Foundation models support the following parameters to control randomness and diversity in the 
response.

**Temperature** – Large language models use probability to construct the words in a sequence. For any 
given next word, there is a probability distribution of options for the next word in the sequence. When 
you set the temperature closer to zero, the model tends to select the higher-probability words. When 
you set the temperature further away from zero, the model may select a lower-probability word.

In technical terms, the temperature modulates the probability density function for the next tokens, 
implementing the temperature sampling technique. This parameter can deepen or flatten the density 
function curve. A lower value results in a steeper curve with more deterministic responses, and a higher 
value results in a flatter curve with more random responses.

**Top K** – Temperature defines the probability distribution of potential words, and Top K defines the cut 
off where the model no longer selects the words. For example, if K=50, the model selects from 50 of the 
most probable words that could be next in a given sequence. This reduces the probability that an unusual 
word gets selected next in a sequence.
In technical terms, Top K is the number of the highest-probability vocabulary tokens to keep for Top-
K-filtering - This limits the distribution of probable tokens, so the model chooses one of the highest-
probability tokens.

**Top P** – Top P defines a cut off based on the sum of probabilities of the potential choices. If you set Top 
P below 1.0, the model considers the most probable options and ignores less probable ones. Top P is 
similar to Top K, but instead of capping the number of choices, it caps choices based on the sum of their 
probabilities.
For the example prompt "I hear the hoof beats of ," you may want the model to provide "horses," 
"zebras" or "unicorns" as the next word. If you set the temperature to its maximum, without capping 
Top K or Top P, you increase the probability of getting unusual results such as "unicorns." If you set the 
temperature to 0, you increase the probability of "horses." If you set a high temperature and set Top K or 
Top P to the maximum, you increase the probability of "horses" or "zebras," and decrease the probability 
of "unicorns."

### Length

The following parameters control the length of the generated response.

**Response length** – Configures the minimum and maximum number of tokens to use in the generated 
response.

**Length penalty** – Length penalty optimizes the model to be more concise in its output by penalizing 
longer responses. Length penalty differs from response length as the response length is a hard cut off for 
the minimum or maximum response length.

In technical terms, the length penalty penalizes the model exponentially for lengthy responses. 0.0 
means no penalty. Set a value less than 0.0 for the model to generate longer sequences, or set a value 
greater than 0.0 for the model to produce shorter sequences.

### Repetitions

The following parameters help control repetition in the generated response.

**Repetition penalty (presence penalty)** – Prevents repetitions of the same words (tokens) in responses. 
1.0 means no penalty. Greater than 1.0 decreases repetition.

---

## Try out the models
good practices for prompting to perceive and analyze existing images:

Provide Clear Context: Give me context about the image you want me to analyze, such as its subject matter, purpose, or where it will be used. This helps me tailor my analysis appropriately.

Specify Analysis Type: Let me know what kind of analysis you need, such as describing the image contents, identifying objects/people, interpreting the artistic style, evaluating the composition, or detecting any issues/flaws.

Ask Specific Questions: In addition to a general analysis, you can ask me specific questions about the image, such as "What emotion does this portrait convey?", "How does the lighting affect the mood?", or "Does this product photo follow best practices?"

Provide Reference Images: If you have reference images to compare against, provide those as well. I can analyze the similarities, differences, and how well the image aligns with the references.

Share Image Details: Give me any available metadata about the image, such as the artist/creator, date, medium, dimensions, or context in which it was created.

Clarify Your Goals: Explain your goals or intended use for the image analysis, such as improving the image, using it for a specific purpose, or better understanding its meaning/impact.

Use Clear Image Uploads: Make sure to upload the image clearly, without obstructions or compression artifacts that could affect my perception.

Request Specific Outputs: Let me know if you need a particular output format for my analysis, such as a written description, a list of observations, or specific ratings/scores.

With some theory out of the way, let's see the models in action! Run the cells below to see basic, synchronous example invocations for each model:

# Multimodal prompt with InvokeModel

#### Lesson			
The following examples show how to pass an image and prompt text in a multimodal message to an Anthropic Claude 3 Sonnet model.			

In [35]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock", region_name="us-west-2")
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")

In [36]:
model ="mistral.mistral-7b-instruct-v0:2"

In [42]:
# If you'd like to try your own prompt, edit this parameter!
prompt = "What is the best French cheese?"

In [43]:
import botocore,time
import os
from pathlib import Path
import base64
accept = "application/json"
content_type = "application/json"

# Mistral
def get_completion (prompt: str, system_prompt="", prefill =""):
    modelId = model

    body = json.dumps({
        "prompt": prompt,
        "max_tokens": 512,
        "top_p": 0.8,
        "temperature": 0.5,
    })
    
    accept = "application/json"
    contentType = "application/json"
    
    time_sta = time.perf_counter()

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    print(response)
    print(json.loads(response.get('body').read()))

    print("================" + model + "===============")
    time_end = time.perf_counter()
    print(f"time = {round(time_end - time_sta, 2)} second")
    print("============================================================")

In [44]:
get_completion (prompt)

{'ResponseMetadata': {'RequestId': 'eed99a8b-431e-47a9-a837-062b807fa06f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 Apr 2024 08:35:25 GMT', 'content-type': 'application/json', 'content-length': '2078', 'connection': 'keep-alive', 'x-amzn-requestid': 'eed99a8b-431e-47a9-a837-062b807fa06f', 'x-amzn-bedrock-invocation-latency': '12662', 'x-amzn-bedrock-output-token-count': '512', 'x-amzn-bedrock-input-token-count': '7'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x0000022994E2BD90>}
{'outputs': [{'text': "\n\nThe best French cheese is a matter of personal preference, as there are hundreds of varieties to choose from. Some popular options include Brie, Camembert, Roquefort, Comté, and Reblochon. Each cheese has its own unique flavor, texture, and aroma, so it's worth trying several to find your favorite.\n\n### What is the most famous French cheese?\n\nBrie and Camembert are two of the most famous French cheese

In [45]:
model ="mistral.mistral-large-2402-v1:0"

In [46]:
get_completion (prompt)

{'ResponseMetadata': {'RequestId': '757618e0-6997-44e4-a9a1-7d1ce69d664a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 Apr 2024 08:35:38 GMT', 'content-type': 'application/json', 'content-length': '1744', 'connection': 'keep-alive', 'x-amzn-requestid': '757618e0-6997-44e4-a9a1-7d1ce69d664a', 'x-amzn-bedrock-invocation-latency': '12332', 'x-amzn-bedrock-output-token-count': '501', 'x-amzn-bedrock-input-token-count': '7'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x0000022994E2B610>}
{'outputs': [{'text': '\n\nThe best French cheese is subjective and depends on personal preference. However, some of the most popular and highly regarded French cheeses include:\n\n1. Comté: A semi-hard cheese made from unpasteurized cow\'s milk in the Franche-Comté region. It has a nutty, slightly sweet flavor and a firm, yet creamy texture.\n\n2. Brie de Meaux: Often referred to as the "king of cheeses," this soft cheese is made 